<div dir="rtl">

## عملکرد کلی بخش اول::
این برنامه برای شناسایی تهدیدات سایبری با استفاده از داده‌های **CAPEC** و **APT** طراحی شده است.
داده‌های **CAPEC** شامل الگوهای حملات هستند که به عبارات کلیدی و توضیحات مرتبط با هر حمله اشاره دارند.
داده‌های **APT** شامل نگاشت این الگوها به گروه‌های تهدیدات (APT) است.
برنامه لاگ‌های **SPLUNK** را پردازش می‌کند و با استفاده از تکنیک‌های پردازش زبان طبیعی (Natural Language Processing)
مانند **spaCy**، حملات مشابه را شبیه‌سازی و تطبیق می‌دهد.

## روش‌های پیاده‌سازی:
1. **بارگذاری داده‌های CAPEC:** داده‌های CAPEC از یک فایل CSV بارگذاری می‌شود. این داده‌ها شامل نام، توضیحات،
    و نمونه‌هایی از حملات هستند که عبارات کلیدی آن‌ها استخراج و ذخیره می‌شود.
2. **بارگذاری نگاشت CAPEC به APT:** نگاشت CAPEC به APT از یک فایل CSV دیگر بارگذاری می‌شود. این نگاشت مشخص می‌کند که
    هر الگوی CAPEC به کدام گروه APT مرتبط است.
3. **پردازش لاگ‌های SPLUNK:** داده‌های SPLUNK که در قالب یک فایل JSON ذخیره شده‌اند، پردازش می‌شوند.
    در این مرحله، هر لاگ با استفاده از **spaCy PhraseMatcher** با عبارات CAPEC تطبیق داده می‌شود.
4. **فیلتر APT:** در صورت نیاز، تنها لاگ‌هایی که شامل APTهای انتخاب شده باشند، نگه داشته می‌شوند.
5. **گزارش‌دهی و بررسی توسط خبره:** گزارش‌هایی برای بررسی توسط کارشناسان امنیتی ایجاد می‌شود. این گزارش شامل
    اطلاعات لاگ‌های پردازش‌شده و برچسب‌های تهدید (benign/مشکوک) است.

## فایل‌های ورودی:
- **CAPEC_CSV:** فایل CSV که شامل داده‌های CAPEC است.
- **APT_CSV:** فایل CSV که شامل نگاشت CAPEC به APT است.
- **SPLUNK_JSON:** فایل JSON که شامل داده‌های لاگ‌های SPLUNK است.

## فایل‌های خروجی:
- **OUTPUT_CSV:** فایل CSV که شامل داده‌های پردازش‌شده است، شامل اطلاعات مختلف لاگ‌ها مانند
    زمان، منبع، IP کاربر، و وضعیت تهدید.
- **EXPERT_REPORT_CSV:** فایل CSV که برای بررسی توسط کارشناسان ارسال می‌شود. این فایل شامل
    اطلاعات لاگ‌ها و ستون‌هایی برای بررسی صحت برچسب‌ها توسط خبره است.

## تفسیر خروجی:
پس از پردازش داده‌ها، در نهایت یک فایل CSV شامل داده‌های پردازش‌شده تولید می‌شود که هر رکورد آن شامل اطلاعات لاگ
و برچسب تهدید (benign/مشکوک) است.
در این فایل، برچسب "benign" نشان‌دهنده عدم شناسایی تهدید است (1 = بی‌خطر)، و اگر CAPECها شبیه‌سازی شوند،
برچسب "benign" به 0 تغییر می‌کند.
همچنین در فایل **EXPERT_REPORT_CSV**، کارشناس امنیتی باید صحت این برچسب‌ها را بررسی کند.

### گزارش تطبیق APT:
پس از پردازش، اگر APTهایی خاص انتخاب شوند، گزارشی از تعداد حملات تطبیق داده‌شده با این APTها تولید می‌شود.


همچنین، امکان فیلتر و گزارش‌دهی
برای بررسی دقیق‌تر توسط کارشناسان امنیتی فراهم می‌شود.
در نهایت، با استفاده از توافق بین برچسب‌های خودکار و برچسب‌های خبره (Cohen's Kappa)، کیفیت برچسب‌ها ارزیابی می‌شود.


In [ ]:
import pandas as pd
import json
import re
import spacy
import logging
from spacy.matcher import PhraseMatcher
from sklearn.metrics import cohen_kappa_score
from typing import Dict, List, Tuple

# تنظیمات اولیه لاگینگ
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# بارگذاری مدل زبان spaCy برای پردازش پیشرفته متن
nlp = spacy.load("en_core_web_sm")

def normalize_text(text: str) -> List[str]:
    """
    نرمال‌سازی متن شامل تبدیل به حروف کوچک، حذف کاراکترهای ناخواسته، لِماتیزه کردن و حذف کلمات توقف.
    """
    doc = nlp(text.lower())
    tokens = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]
    return tokens

def load_capec_data(capec_csv: str) -> Tuple[Dict[str, List[str]], pd.DataFrame]:
    """
    بارگذاری داده‌های CAPEC و ایجاد لیستی از عبارات کلیدی برای هر الگو.
    """
    df = pd.read_csv(capec_csv)
    capec_phrases = {}
    capec_details = []

    for _, row in df.iterrows():
        capec_id = f"CAPEC-{row['ID']}" if not str(row['ID']).startswith('CAPEC-') else row['ID']

        # استخراج عبارات کلیدی از فیلدهای مرتبط
        phrases = []
        for field in ['Name', 'Description', 'Example Instances', 'Alternate Terms']:
            text = str(row.get(field, ''))
            if text.lower() != 'nan' and text.strip():
                # تقسیم متن و نرمال‌سازی عبارات
                raw_phrases = [phrase.strip() for phrase in re.split(r'[;,.]', text) if phrase.strip()]
                for phrase in raw_phrases:
                    # برای هر عبارت، نسخه‌های نرمال‌شده را استخراج می‌کنیم (می‌توان این بخش را بر اساس نیاز تغییر داد)
                    phrases.append(phrase.lower())
        # حذف عبارات تکراری و عبارات کوتاه
        phrases = list(set([p for p in phrases if len(p) > 3]))
        capec_phrases[capec_id] = phrases
        capec_details.append({
            'CAPEC-ID': capec_id,
            'Name': row['Name'],
            'Severity': row.get('Typical Severity', 'Medium'),
            'Description': row.get('Description', ''),
            'Keywords': phrases
        })

    return capec_phrases, pd.DataFrame(capec_details)

def load_apt_mapping(apt_csv: str) -> pd.DataFrame:
    """
    بارگذاری نگاشت CAPEC به APT از فایل CSV.
    فرض بر این است که فایل شامل ستون‌های 'CAPEC-ID' و 'APT Group' می‌باشد.
    """
    apt_df = pd.read_csv(apt_csv)
    return apt_df

def process_logs(splunk_data: List[Dict], capec_phrases: Dict, apt_mapping: pd.DataFrame,
                 selected_apts: List[str] = None) -> pd.DataFrame:
    """
    پردازش لاگ‌های SPLUNK و تطبیق با عبارات CAPEC با استفاده از spaCy PhraseMatcher.
    علاوه بر آن، نگاشت CAPEC به APT انجام می‌شود.
    """
    print(f"✅ تعداد لاگ‌های ورودی از JSON: {len(splunk_data)}")  # مرحله اول بررسی تعداد ورودی

    matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
    patterns = {capec_id: [nlp(text) for text in phrases] for capec_id, phrases in capec_phrases.items()}

    for capec_id, pattern_list in patterns.items():
        matcher.add(capec_id, pattern_list)

    processed_data = []
    skipped_logs = 0

    for idx, log in enumerate(splunk_data):
        try:
            # استخراج فیلدهای مرتبط از لاگ با مقدار پیش‌فرض
            features = {
                '_time': log.get('_time', ''),
                'host': log.get('host', ''),
                'source': log.get('source', ''),
                'sourcetype': log.get('sourcetype', ''),
                'status': log.get('status', 0),
                'bytes': log.get('bytes', 0),
                'clientip': log.get('clientip', ''),
                'url': log.get('url', ''),
                'level': log.get('level', ''),
                'component': log.get('component', ''),
                'message': log.get('message', '').lower() if log.get('message') else ''
            }

            # پردازش CAPEC
            doc = nlp(features['message'])
            matches = matcher(doc)
            matched_capecs = list(set([nlp.vocab.strings[match_id] for match_id, _, _ in matches]))
            features['matched_capecs'] = ', '.join(matched_capecs)
            features['benign'] = 1 if not matched_capecs else 0

            # نگاشت CAPEC به APT
            apt_list = []
            for capec in matched_capecs:
                apt_rows = apt_mapping[apt_mapping['CAPEC-ID'] == capec]
                if not apt_rows.empty:
                    apt_list.extend(apt_rows['APT Group'].tolist())
            apt_list = list(set(apt_list))
            features['matched_apts'] = ', '.join(apt_list)
            features['expert_validation'] = None

            processed_data.append(features)

        except Exception as e:
            print(f"⚠️ خطا در پردازش لاگ شماره {idx}: {e}")
            skipped_logs += 1
            continue

    df = pd.DataFrame(processed_data)
    print(f"📊 تعداد لاگ‌های پردازش شده نهایی: {len(df)}")
    print(f"⚠️ تعداد لاگ‌های رد شده به دلیل خطا: {skipped_logs}")

    # -----------------------
    # اگر APT خاصی انتخاب شده باشد، فیلتر کنیم
    if selected_apts is not None and len(selected_apts) > 0:
        def filter_apt(row):
            if row['matched_apts']:
                apt_set = set([apt.strip() for apt in row['matched_apts'].split(',')])
                return bool(apt_set.intersection(set(selected_apts)))
            return False

        initial_count = len(df)
        df = df[df.apply(filter_apt, axis=1)]
        filtered_count = len(df)
        print(f"📊 فیلتر زیرمجموعه APT: از {initial_count} رکورد، {filtered_count} رکورد انتخاب شدند.")
        # گزارش تعداد هر APT انتخاب شده
        apt_summary = {}
        for apt in selected_apts:
            count = df['matched_apts'].str.contains(apt, regex=False).sum()
            apt_summary[apt] = count
        print("🔍 گزارش انتخاب زیرمجموعه APT:", apt_summary)

    return df


def generate_expert_report(df: pd.DataFrame, output_file: str):
    """
    ایجاد گزارش برای بررسی توسط خبره.
    """
    report = df[[
        '_time', 'host', 'source', 'sourcetype', 'status', 'bytes',
        'clientip', 'url', 'level', 'component', 'message', 'matched_capecs',
        'matched_apts', 'benign'
    ]].copy()
    report['expert_capecs'] = ''
    report['expert_benign'] = ''
    report.to_csv(output_file, index=False)
    logging.info(f"گزارش خبره در فایل {output_file} ذخیره شد.")

def calculate_irr(auto_labels, expert_labels):
    """محاسبه توافق بین برچسب‌های خودکار و خبره با استفاده از Cohen's Kappa."""
    return cohen_kappa_score(auto_labels, expert_labels)

if __name__ == "__main__":
    # پارامترها
    CAPEC_CSV = 'capec_dataset.csv'         # فایل CAPEC
    APT_CSV = 'capec_apt_mapping.csv'         # فایل نگاشت CAPEC به APT
    SPLUNK_JSON = "splunk_logs_sample.json"   # فایل لاگ‌های SPLUNK
    OUTPUT_CSV = 'labeled_threats.csv'
    EXPERT_REPORT_CSV = 'expert_review.csv'

    # بارگذاری داده‌های CAPEC و نگاشت APT
    capec_phrases, capec_df = load_capec_data(CAPEC_CSV)
    apt_mapping = load_apt_mapping(APT_CSV)

    with open(SPLUNK_JSON, 'r') as f:
        splunk_data = json.load(f)

    selected_apts = None  # یا می‌توانید لیست APTهای مورد نظر را وارد کنید
    processed_df = process_logs(splunk_data, capec_phrases, apt_mapping, selected_apts)

    # ذخیره دیتاست اولیه
    processed_df.to_csv(OUTPUT_CSV, index=False)
    logging.info(f"فایل خروجی برچسب‌گذاری شده در {OUTPUT_CSV} ذخیره شد.")

    # ایجاد گزارش برای بررسی خبره
    generate_expert_report(processed_df, EXPERT_REPORT_CSV)
    logging.info(f'✅ مرحله ۱ تکمیل شد! لطفاً فایل {EXPERT_REPORT_CSV} را به خبره ارسال کنید.')

    # پس از دریافت فایل اصلاح‌شده توسط خبره:
    # expert_reviewed = pd.read_csv('expert_reviewed.csv')
    # kappa = calculate_irr(processed_df['benign'], expert_reviewed['expert_benign'])
    # logging.info(f'میزان توافق (IRR): {kappa:.2f}')


✅ تعداد لاگ‌های ورودی از JSON: 209
📊 تعداد لاگ‌های پردازش شده نهایی: 209
⚠️ تعداد لاگ‌های رد شده به دلیل خطا: 0


<hr>
<div dir="rtl">

<h2> مراحل کار بخش دوم پروژه</h2>

<h3>1️⃣ بارگذاری داده‌های برچسب‌خورده</h3>
<ul>
  <li>ورودی: فایل <code>labeled_threats.csv</code> (خروجی مرحله اول)</li>
  <li>شامل اطلاعات لاگ‌ها و ستون‌های: <code>matched_capecs</code>، <code>matched_apts</code>، <code>benign</code>، <code>expert_validation</code></li>
  <li>برچسب نهایی داده‌ها در ستون <code>final_label</code> قرار می‌گیرد</li>
</ul>

<h3>2️⃣ تقسیم داده‌ها به آموزش و تست</h3>
<ul>
  <li>با <code>train_test_split</code> داده‌ها به دو بخش آموزش (70٪) و تست (30٪) تقسیم می‌شوند</li>
  <li>از <code>stratify</code> برای حفظ نسبت تهدید و عادی استفاده می‌شود</li>
</ul>

<h3>3️⃣ پیش‌پردازش ویژگی‌ها</h3>
<ul>
  <li><strong>ویژگی متنی:</strong> ستون <code>message</code> → پردازش با <code>TF-IDF</code> و n-gram</li>
  <li><strong>ویژگی عددی:</strong> ستون‌های <code>status</code>، <code>bytes</code> → مقیاس‌بندی با <code>RobustScaler</code></li>
  <li><strong>ویژگی دسته‌ای:</strong> ستون‌های <code>level</code>، <code>component</code>، <code>host</code> → کدگذاری با <code>OneHotEncoder</code></li>
</ul>

<h3>4️⃣ مدل‌سازی</h3>
<ul>
  <li>مدل: <code>RandomForestClassifier</code> (مدل نظارت شده)</li>
  <li>تنظیم پارامترها با <code>GridSearchCV</code> برای انتخاب بهترین مقدار <code>n_estimators</code> و <code>max_depth</code></li>
</ul>

<h3>5️⃣ ارزیابی مدل</h3>
<ul>
  <li>ارزیابی روی مجموعه تست با متریک‌های:</li>
  <ul>
    <li><code>classification_report</code>: precision, recall, f1-score</li>
    <li><code>confusion_matrix</code>: تعداد پیش‌بینی درست و غلط</li>
    <li><strong>Recall برای تهدیدها</strong>: مهم‌ترین معیار</li>
  </ul>
</ul>

<h3>6️⃣ گزارش تهدیدات پیشرفته (APT)</h3>
<ul>
  <li>بررسی لاگ‌های پیش‌بینی‌شده به‌عنوان تهدید (label=0)</li>
  <li>تحلیل ستون <code>matched_capecs</code> و نگاشت آن‌ها به APT Groupها</li>
  <li>نمایش نوع و تعداد تهدیدات</li>
</ul>

<h3>7️⃣ ذخیره مدل آموزش‌دیده</h3>
<ul>
  <li>مدل آموزش‌دیده در فایل <code>apt_aware_supervised.pkl</code> ذخیره می‌شود</li>
</ul>

<hr>

<h2>🟢 ورودی‌ها</h2>
<ul>
  <li><code>labeled_threats.csv</code>: داده‌های لاگ پردازش شده و برچسب‌خورده</li>
  <li><code>capec_apt_mapping.csv</code>: نگاشت CAPEC به APT Group</li>
</ul>

<h2>🔵 خروجی‌ها</h2>
<ul>
  <li>گزارش ارزیابی مدل (precision, recall, confusion matrix)</li>
  <li>گزارش تهدیدات پیشرفته APT</li>
  <li>فایل مدل ذخیره شده: <code>apt_aware_supervised.pkl</code></li>
</ul>

<hr>

<h2>📄 نحوه خواندن و تفسیر خروجی‌ها</h2>

<h3>1️⃣ مدل ذخیره شده</h3>
<p>مدل ذخیره شده با <code>joblib</code> بارگذاری می‌شود:</p>
<pre><code>import joblib
model = joblib.load('apt_aware_supervised.pkl')</code></pre>
<p>سپس می‌توانید داده‌های جدید را مشابه قبل پردازش و پیش‌بینی کنید.</p>

<h3>2️⃣ گزارش ارزیابی مدل</h3>
<p>شامل:</p>
<ul>
  <li><strong>Support:</strong> تعداد واقعی نمونه‌های هر کلاس</li>
  <li><strong>Precision:</strong> دقت پیش‌بینی‌ها</li>
  <li><strong>Recall:</strong> مخصوصاً برای تهدیدها بسیار مهم است</li>
  <li><strong>Confusion Matrix:</strong> نمایش پیش‌بینی درست و غلط</li>
</ul>

<h3>3️⃣ گزارش تهدیدات پیشرفته (APT)</h3>
<p>نشان می‌دهد مدل چه نوع تهدیداتی (APT Groupها) را شناسایی کرده و چند مورد از هر نوع وجود داشته.</p>


In [ ]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix, recall_score
from sklearn.model_selection import train_test_split, GridSearchCV
import joblib
import logging

# تنظیمات لاگینگ
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# ---------------------------
def load_data(file_path: str) -> pd.DataFrame:
    """بارگذاری داده‌ها و ایجاد برچسب نهایی"""
    df = pd.read_csv(file_path)

    if 'expert_benign' in df.columns:
        logging.info("✅ استفاده از برچسب‌های خبره")
        df['final_label'] = np.where(
            df['expert_benign'].notna(),
            df['expert_benign'].astype(int),
            df['benign']
        )
    else:
        logging.warning("⚠️ استفاده از برچسب‌های خودکار")
        df['final_label'] = df['benign']

    logging.info("\n📊 آمار کل داده:")
    logging.info(df['final_label'].value_counts())
    return df

# ---------------------------
def get_preprocessor():
    """
    پیش‌پردازش ویژگی‌ها شامل بخش متنی (با TF-IDF) و عددی/دسته‌ای.
    """
    text_processor = Pipeline([
        ('tfidf', TfidfVectorizer(
            max_features=3000,
            ngram_range=(1, 2),
            stop_words='english',
            sublinear_tf=True,
            analyzer='word'
        ))
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ('text', text_processor, 'message'),
            ('num', RobustScaler(), ['status', 'bytes']),
            ('cat', OneHotEncoder(
                handle_unknown='infrequent_if_exist',
                sparse_output=False,
                min_frequency=10
            ), ['level', 'component', 'host'])
        ],
        remainder='drop'
    )
    return preprocessor

# ---------------------------
def train_model_supervised(X_train: pd.DataFrame, y_train: pd.Series) -> Pipeline:
    """آموزش مدل نظارت شده با استفاده از GridSearchCV برای تنظیم پارامترها"""
    preprocessor = get_preprocessor()

    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', RandomForestClassifier(random_state=42, n_jobs=-1))
    ])

    param_grid = {
        'classifier__n_estimators': [100, 200],
        'classifier__max_depth': [None, 10, 20],
    }

    grid_search = GridSearchCV(
        pipeline,
        param_grid,
        cv=5,
        scoring='f1_weighted',
        verbose=1,
        n_jobs=-1
    )

    logging.info("\n🚀 شروع آموزش مدل نظارت شده با GridSearchCV:")
    grid_search.fit(X_train, y_train)
    logging.info(f"بهترین پارامترها: {grid_search.best_params_}")

    return grid_search.best_estimator_

# ---------------------------
def analyze_apts(threat_data: pd.DataFrame, capec_apt_map: pd.DataFrame) -> dict:
    """تحلیل APT با استفاده از نگاشت CAPEC به APT"""
    apt_counts = {}
    for _, row in threat_data.iterrows():
        if pd.isna(row.get('matched_capecs')):
            continue
        capecs = [c.strip() for c in row['matched_capecs'].split(',')]
        for capec in capecs:
            apts = capec_apt_map[capec_apt_map['CAPEC-ID'] == capec]['APT Group']
            for apt in apts:
                apt_counts[apt] = apt_counts.get(apt, 0) + 1
    return dict(sorted(apt_counts.items(), key=lambda x: x[1], reverse=True))

# ---------------------------
def evaluate_model(model: Pipeline, X_test: pd.DataFrame, y_test: pd.Series, capec_apt_map: pd.DataFrame):
    """ارزیابی مدل نظارت شده و چاپ گزارش عملکرد"""
    y_pred = model.predict(X_test)

    print("\n📊 آمار داده‌های تست:")
    print(y_test.value_counts())

    report = classification_report(
        y_test,
        y_pred,
        target_names=['تهدید (0)', 'عادی (1)'],
        digits=4
    )
    cm = confusion_matrix(y_test, y_pred)
    threat_recall = recall_score(y_test, y_pred, pos_label=0)

    print("\n📊 گزارش عملکرد:")
    print(report)
    print("\n🎯 ماتریس درهم‌ریختگی:")
    print(cm)
    print(f"\n📈 Recall برای تهدیدها: {threat_recall:.2%}")

    # گزارش APT (در صورتی که ستون matched_capecs موجود باشد)
    if 'matched_capecs' in X_test.columns:
        X_test = X_test.copy()
        X_test['predicted_label'] = y_pred
        threat_logs = X_test[X_test['predicted_label'] == 0]
        print("\n🔍 گزارش تهدیدات پیشرفته (APT) برای لاگ‌های خطرناک:")
        if not threat_logs.empty:
            apt_report = analyze_apts(threat_logs, capec_apt_map)
            for apt, count in apt_report.items():
                print(f"- {apt}: {count} مورد")
        else:
            print("⚠️ هیچ تهدیدی شناسایی نشد!")
    else:
        print("ستون matched_capecs در داده‌ها موجود نیست؛ بنابراین گزارش APT تولید نمی‌شود.")

# ---------------------------
if __name__ == "__main__":
    INPUT_DATA = "labeled_threats.csv"
    CAPEC_APT_MAP = "capec_apt_mapping.csv"

    # بارگذاری داده‌ها
    df = load_data(INPUT_DATA)

    # تقسیم داده‌ها به آموزش و تست (70/30)
    X = df.drop(columns=['final_label'])
    y = df['final_label']

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42, stratify=y
    )

    print("\n📊 آمار داده‌های آموزش:")
    print(y_train.value_counts())
    print("➖➖➖")
    print("📊 آمار داده‌های تست:")
    print(y_test.value_counts())
    print("➖➖➖")

    capec_apt_map = pd.read_csv(CAPEC_APT_MAP)

    # آموزش مدل
    model = train_model_supervised(X_train, y_train)

    # ارزیابی مدل
    evaluate_model(model, X_test, y_test, capec_apt_map)

    # ذخیره مدل
    joblib.dump(model, 'apt_aware_supervised.pkl')
    print("\n💾 مدل ذخیره شد.")



📊 آمار داده‌های آموزش:
final_label
1    85
0    61
Name: count, dtype: int64
➖➖➖
📊 آمار داده‌های تست:
final_label
1    36
0    27
Name: count, dtype: int64
➖➖➖
Fitting 5 folds for each of 6 candidates, totalling 30 fits

📊 آمار داده‌های تست:
final_label
1    36
0    27
Name: count, dtype: int64

📊 گزارش عملکرد:
              precision    recall  f1-score   support

   تهدید (0)     1.0000    1.0000    1.0000        27
    عادی (1)     1.0000    1.0000    1.0000        36

    accuracy                         1.0000        63
   macro avg     1.0000    1.0000    1.0000        63
weighted avg     1.0000    1.0000    1.0000        63


🎯 ماتریس درهم‌ریختگی:
[[27  0]
 [ 0 36]]

📈 Recall برای تهدیدها: 100.00%

🔍 گزارش تهدیدات پیشرفته (APT) برای لاگ‌های خطرناک:
- China-based APT groups: 2 مورد
- Chimera, APT-C-23, APT3: 1 مورد

💾 مدل ذخیره شد.


<hr>
<div dir="rtl">
<h2> مدل تمام تهدیدات رو درست پیش بینی کرده که به دلایل زیر هست:</h2>
تعداد کم داده‌ها و یکنواخت بودن:
فقط 209 لاگ داریم.
داده‌ها خیلی متنوع نیستن و الگوی خاصی به وضوح بین تهدید و عادی وجود داره.

همچنین با stratify split نسبت داده‌های تهدید و عادی در آموزش و تست حفظ شده و تفاوت‌ها خیلی واضح هستند.

4️⃣ مدل RandomForest قدرت بالایی در جدا کردن داده‌های ساده داره:
RandomForest مدل قدرتمندیه، وقتی داده‌ها الگوهای واضح و بدون نویز داشته باشن، به راحتی می‌تونه بدون خطا پیش‌بینی کنه.

